In [2]:
!pip install yfinance

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 10.5 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 32.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.9/152.9 kB 7.7 MB/s eta 0:00:00
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15548 sha256=c159cc842af3606a6517c0f5edbe057ff56b8d2c698855129b948df505dc5e0e
  Stored in directory: /home/jovyan/.cache/pip/wheels/42/d6/84/bf57a755f4569494cd00de4bb46ef064874823f4d19c82e960
  Created wheel for peewee: filename=peewee-3.18.3-cp311-cp311-linux_x86_64.whl size=300929 sha256=f9c76bfc998b22ed51dcfa3161bff83b

In [1]:
import yfinance as yf
import pandas as pd
from datetime import date, timedelta
from pathlib import Path
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# 1. PEŁNA LISTA GRUP I TICKERÓW
GROUPS = {
    "USA": ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'TSLA', 'BRK-B', 'UNH', 'V', 'JNJ', 'WMT', 'JPM', 'MA', 'PG', 'AVGO', 'HD', 'CVX', 'ORCL', 'ABBV', 'KO', 'PEP', 'COST', 'BAC', 'ADBE'],
    "INDEX": ['^GSPC', '^IXIC', '^DJI', '^RUT', '^VIX', '^FTSE', '^GDAXI', '^FCHI', '^N225', '^HSI', '^STOXX50E', 'QQQ', 'SPY', 'IWM', 'EEM', 'VWO', 'VEA', 'VNQ', 'GLD', 'SLV', 'DBC', 'TLT', 'HYG', 'VTI', 'VXUS'],
    "GPW": ['CDR.WA', 'PKO.WA', 'PKN.WA', 'KGH.WA', 'PZU.WA', 'PEO.WA', 'ALE.WA', 'LPP.WA', 'DNP.WA', 'PGE.WA', 'OPL.WA', 'SPL.WA', 'JSW.WA', 'ACP.WA', 'KRU.WA', 'MBK.WA', 'CPS.WA', 'TPE.WA', 'KTY.WA', 'ATT.WA', 'ASB.WA', 'BDX.WA', 'GPW.WA', '11B.WA', 'CCC.WA'],
    "CRYPTO": ['BTC-USD', 'ETH-USD', 'SOL-USD', 'BNB-USD', 'XRP-USD', 'ADA-USD', 'DOGE-USD', 'AVAX-USD', 'DOT-USD', 'TRX-USD', 'LINK-USD', 'SHIB-USD', 'LTC-USD', 'DAI-USD', 'BCH-USD', 'ATOM-USD', 'UNI7083-USD', 'XLM-USD', 'XMR-USD', 'ETC-USD', 'FIL-USD', 'HBAR-USD', 'NEAR-USD'],
    "COMMODITIES": ['GC=F', 'SI=F', 'CL=F', 'NG=F', 'HG=F', 'ZC=F', 'ZW=F', 'KC=F', 'CC=F', 'CT=F', 'OJ=F', 'PL=F', 'PA=F', 'SB=F', 'SOYB', 'WEAT', 'CORN', 'WOOD', 'LIT', 'REMX', 'TAN', 'FAN', 'PICK', 'URA']
}

# Spłaszczamy listę do pobierania
ALL_TICKERS = [ticker for group in GROUPS.values() for ticker in group]

# 2. KONFIGURACJA ŚCIEŻEK I SPARK
BRONZE_STOCKS_PATH = Path("/home/jovyan/work/data/bronze/incremental_stocks")
BRONZE_STOCKS_PATH.mkdir(parents=True, exist_ok=True)

spark = SparkSession.builder \
    .appName("Stocks_Incremental_125_Tickers") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.6.0") \
    .getOrCreate()

DB_CONF = {
    "url": "jdbc:postgresql://postgres_dw:5432/currency_db",
    "user": "admin",
    "password": "password123",
    "driver": "org.postgresql.Driver"
}

# 3. SENSOR: Sprawdzamy najnowszą datę w Postgresie
def get_last_trade_date():
    try:
        df = spark.read.format("jdbc").options(**DB_CONF).option("dbtable", "f_stock_prices").load()
        return df.select(F.max("trade_date")).collect()[0][0]
    except Exception:
        return date(2025, 1, 1)

last_db_date = get_last_trade_date()
start_fetch = last_db_date + timedelta(days=1)
today = date.today()

if start_fetch >= today:
    print(f" Dane giełdowe są aktualne. Ostatnia data: {last_db_date}")
else:
    print(f" Wykryto brakujące dni ({start_fetch} do {today}). Pobieram 125 tickerów...")

    # 4. POBIERANIE (Bronze - Fizyczne pliki CSV)
    all_new_dfs = []
    
    # Pobieramy dane w małych paczkach (per grupa), aby uniknąć limitów API
    for group_name, tickers in GROUPS.items():
        print(f"--- Grupa: {group_name} ---")
        # Pobieramy dane zbiorczo dla grupy (szybsze niż pojedynczo)
        data = yf.download(tickers, start=start_fetch, progress=False, group_by='ticker')
        
        for ticker in tickers:
            try:
                # Wyciągamy dane dla konkretnego tickera z pobranego obiektu
                if len(tickers) > 1:
                    t_data = data[ticker].dropna().reset_index()
                else:
                    t_data = data.dropna().reset_index()

                if not t_data.empty:
                    t_data['ticker'] = ticker
                    t_data['group_name'] = group_name
                    
                    # Zapis fizyczny CSV (Bronze)
                    file_name = BRONZE_STOCKS_PATH / f"{ticker}_{today}.csv"
                    t_data.to_csv(file_name, index=False)
                    
                    all_new_dfs.append(t_data)
            except Exception as e:
                print(f"   Pominąłem {ticker}: brak nowych danych lub błąd.")

    # 5. ŁADOWANIE DO POSTGRESA (Silver - Append)
    if all_new_dfs:
        combined_pandas_df = pd.concat(all_new_dfs)
        spark_df = spark.createDataFrame(combined_pandas_df)

        # Standaryzacja kolumn pod Twoją strukturę tabeli
        final_stocks = spark_df.select(
            F.col("ticker"),
            F.col("Date").cast("date").alias("trade_date"),
            F.col("Open").cast("decimal(14,4)").alias("open_price"),
            F.col("High").cast("decimal(14,4)").alias("high_price"),
            F.col("Low").cast("decimal(14,4)").alias("low_price"),
            F.col("Close").cast("decimal(14,4)").alias("close_price"),
            F.col("Volume").cast("bigint").alias("volume")
        )

        print(f" Zapisuję {final_stocks.count()} nowych wierszy do bazy...")
        
        final_stocks.write \
            .format("jdbc") \
            .options(**DB_CONF) \
            .option("dbtable", "f_stock_prices") \
            .mode("append") \
            .save()
            
        print(f" GOTOWE! Twoja baza została zasilona nowymi danymi z {len(all_new_dfs)} tickerów.")
    else:
        print(" Brak nowych zamkniętych sesji giełdowych do pobrania.")

✅ Dane giełdowe są aktualne. Ostatnia data: 2026-01-05
